In [1]:
# this is fully stolen from the internet, I just changed it to use plotly
from __future__ import print_function, division
import pandas as pd
import numpy_financial as npf
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from datetime import date, datetime

from ipywidgets import widgets

In [2]:
interest   = 0.08
fifteen    = 15
thirty     = 30
ppy        = 12
mortgage  = 460000
term = 15
start_date = date(2023,9,1)

extra_payment = 0

In [3]:
#  mortgage amount, interest, term, and extra payment
def make_payment_table(m, i, t, e):
    initial_payment  = -1 * npf.pmt(i/ppy, t*ppy, m)
    initial_ipayment = -1 * npf.ipmt(i/ppy, 1, t*ppy, m)
    initial_ppayment = -1 * npf.ppmt(i/ppy, 1, t*ppy, m)
    
    rng = pd.Series(range(1,1+12*t))
    rng.name = "Payment Date"
    df = pd.DataFrame(
        index=rng,
        columns=['Total Payment',
                 'Interest',
                 'Intereste',                 
                 'Principal',
                 'Principale',                 
                 'Additional Payment',
                 'Ending Balance w/ Extra',
                 'Ending Balance'],
        dtype='float')

    #df.reset_index(inplace=True)
    df.index.name = "Period"
    
    
    # First row (Think of it as the seed for the whole thing)

    period = 1

    initial_seed = {
        'Total Payment'  : initial_payment,
        'Interest'       : initial_ipayment,
        'Principal'      : initial_ppayment,
        'Intereste'       : initial_ipayment,
        'Principale'      : initial_ppayment + e,
        'Additional Payment' : e,
        'Ending Balance w/ Extra' : m - initial_payment - e,
        'Ending Balance'          : m - initial_payment
    }


    df.at[period, initial_seed.keys()] = initial_seed.values()
    df = df.round(2)



    #Populate the rest of the payments table

    for period in range(2, len(df) + 1):
        previous_tp   = df.loc[period - 1, 'Total Payment']
        previous_pr   = df.loc[period - 1, 'Principal']
        previous_end  = df.loc[period - 1, 'Ending Balance']
        previous_ende = df.loc[period - 1, 'Ending Balance w/ Extra']

        period_interest   = previous_end  * i / ppy
        period_intereste  = previous_ende * i / ppy
        period_principal  = initial_payment - period_interest
        period_principale = initial_payment - period_intereste + e
        
        ending_drafte     = previous_ende - period_principale - e
        ending_balancee   = 0 if ending_drafte <= 0 else ending_drafte
        ending_draft      = previous_end - period_principal
        ending_balance    = 0 if ending_draft <= 0 else ending_draft

        row = {
            'Total Payment'  : initial_payment,
            'Interest'       : period_interest,
            'Principal'      : period_principal,
            'Intereste'       : period_intereste,
            'Principale'      : period_principale,
            'Additional Payment' : e,
            'Ending Balance w/ Extra' : ending_balancee,
            'Ending Balance'     : ending_balance
        }

        #df.at[period, row.keys()] = row.values()
        df.loc[period] = pd.Series(row)

    df['Payment Date'] = rng
    return df, initial_payment

In [4]:
df, ip = make_payment_table(400000, 0.08, 15, 1000)



In [5]:
default_end_date = df[df['Ending Balance'] == 0]['Payment Date'].min()
extra_end_date   = df[df['Ending Balance w/ Extra'] == 0]['Payment Date'].min()

round((default_end_date - extra_end_date) / 12, 2)

7.17

In [6]:
df

,Total Payment,Interest,Intereste,Principal,Principale,Additional Payment,Ending Balance w/ Extra,Ending Balance,Payment Date
Period,,,,,,,,,
1,3822.610000,2666.670000,2666.670000,1155.940000,2155.940000,1000.0,395177.390000,396177.390000,2.0
2,3822.608337,2641.182600,2634.515933,1181.425737,2188.092404,1000.0,391989.297596,394995.964263,3.0
3,3822.608337,2633.306428,2613.261984,1189.301909,2209.346353,1000.0,388779.951243,393806.662354,4.0
4,3822.608337,2625.377749,2591.866342,1197.230588,2230.741996,1000.0,385549.209247,392609.431765,5.0
5,3822.608337,2617.396212,2570.328062,1205.212126,2252.280276,1000.0,382296.928971,391404.219640,6.0
...,...,...,...,...,...,...,...,...,...
176,3822.608337,68.418921,0.000000,3754.189417,4822.608337,1000.0,0.000000,6508.648703,177.0
177,3822.608337,43.390991,0.000000,3779.217346,4822.608337,1000.0,0.000000,2729.431357,178.0
178,3822.608337,18.196209,0.000000,3804.412128,4822.608337,1000.0,0.000000,0.000000,179.0


In [7]:
df["Interest"].sum() - df["Intereste"].sum()

146704.78766066203

In [8]:
melted = pd.melt(df, id_vars=['Payment Date'], value_vars=['Ending Balance', 
                                                          'Ending Balance w/ Extra'])

fig = px.line(melted, x='Payment Date', y='value', color='variable', width=800, height=800)
fig.add_vline(x=extra_end_date, line_dash='dot', line_color="black")
fig.add_vrect(
    x0=extra_end_date,
    x1=default_end_date,
    fillcolor="green",
    opacity=0.2,
    annotation_position="top left",
    annotation_text="{0} Months Earlier endate".format((default_end_date - extra_end_date))
)

fig.update_layout(
    title="Mortgage Payment Timeline at ${0} per Month".format(round(ip,2)),
    xaxis_title="Period",
    yaxis_title="Principal Remaining",
    xaxis_range=[1,360],
    yaxis_range=[0,600000],
    legend=dict(
        title_text='Balance Variation',
        yanchor='bottom',
        y=0.05,
        xanchor='left',
        x=0.05)
)

g = go.FigureWidget(fig)

In [9]:
loan_amount = widgets.IntSlider(
    value=400000,
    min=100000,
    max=1000000,
    step=10000,
    description="Loan Size",
    continous_update=True
)

extra_payment = widgets.IntSlider(
    value=500,
    min=0,
    max=2000,
    step=10,
    description="Extra Payment",
    continous_update=True
)

interest_rate = widgets.FloatSlider(
    value = 0.08,
    min=0.01,
    max=0.1,
    step=0.005,
    description="Interest Rate",
    readout_format='.3f',
    continous_update=True
)

term_select = widgets.RadioButtons(
    options=[15, 30],
    value=15, # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Term Length',
)

def modify_plot(dummy):
    df, ip = make_payment_table(loan_amount.value, interest_rate.value, term_select.value, extra_payment.value)
    
    default_end_date = df[df['Ending Balance'] == 0]['Payment Date'].min()
    extra_end_date   = df[df['Ending Balance w/ Extra'] == 0]['Payment Date'].min()
    total_interest = df["Interest"].sum()
    total_intereste = df["Intereste"].sum()
    savings = total_interest - df["Intereste"].sum()
    
    with g.batch_update():
        g.data[0].x  = df["Payment Date"]
        g.data[0].y  = df["Ending Balance"]
        
        g.data[1].x  = df["Payment Date"]
        g.data[1].y  = df["Ending Balance w/ Extra"]
        
        g.layout.shapes[0].x0 = extra_end_date
        g.layout.shapes[0].x1 = extra_end_date
        
        g.layout.shapes[1].x0 = extra_end_date
        g.layout.shapes[1].x1 = default_end_date
        
        g.layout.annotations[0].x = extra_end_date
        g.layout.annotations[0].text = "{0} months earlier endate and ${1} Total Interest".format(default_end_date - extra_end_date, round(total_intereste,2))
        
        g.layout.title = "Mortgage Payment Timeline at {0} per Month with {1} default total interest paid".format(round(ip,2), round(total_interest,2))
    


#g.layout.shapes = []
#g.layout.annotations = []

loan_amount.observe(modify_plot, names="value")
extra_payment.observe(modify_plot, names="value")
interest_rate.observe(modify_plot, names="value")
term_select.observe(modify_plot, names="value")

container = widgets.VBox(children=[widgets.HBox(children=[loan_amount, interest_rate]),
                                   widgets.HBox(children=[term_select, extra_payment])])

widgets.VBox([container,g])   

In [ ]:
g.layout.title

In [ ]:
g.layout.shapes

In [ ]:
g.data[0].x  = df["Payment Date"]
g.data[0].y = df["Ending Balance"]

In [ ]:
initial_payment  = -1 * npf.pmt(interest/ppy, fifteen*ppy, mortgage)
initial_ipayment = -1 * npf.ipmt(interest/ppy, 1, fifteen*ppy, mortgage)
initial_ppayment = -1 * npf.ppmt(interest/ppy, 1, fifteen*ppy, mortgage)

In [ ]:
fig2 = px.line(melted[1:5], x='Payment Date', y='value', color='variable')
go.FigureWidget(fig2)
fig2

In [ ]:
fig2.data[0].y = [1,2,3]

In [ ]:
rng = pd.date_range(start_date, periods=fifteen*ppy, freq='MS')
rng.name = "Payment Date"
df = pd.DataFrame(
    index=rng,
    columns=['Total Payment',
             'Interest',
             'Principal',
             'Additional Payment',
             'Ending Balance'],
    dtype='float')

df.reset_index(inplace=True)
df.index += 1
df.index.name = "Period"

In [ ]:
# First row (Think of it as the seed for the whole thing)

period = 1

initial_seed = {
    'Total Payment'  : initial_payment,
    'Interest'       : initial_ipayment,
    'Principal'      : initial_ppayment,
    'Additional Payment' : extra_payment,
    'Ending Balance'     : mortgage - initial_payment
}


df.at[period, initial_seed.keys()] = initial_seed.values()
df = df.round(2)



#Populate the rest of the payments table

for period in range(2, len(df) + 1):
    previous_tp  = df.loc[period - 1, 'Total Payment']
    previous_pr  = df.loc[period - 1, 'Principal']
    previous_end = df.loc[period - 1, 'Ending Balance']
    
    period_interest  = previous_end * interest / ppy
    period_principal = initial_payment - period_interest
    ending_draft     = previous_end - period_principal - extra_payment
    ending_balance   = 0 if ending_draft <= 0 else ending_draft
    
    row = {
        'Total Payment'  : initial_payment,
        'Interest'       : period_interest,
        'Principal'      : period_principal,
        'Additional Payment' : extra_payment,
        'Ending Balance'     : ending_balance
    }

    #df.at[period, row.keys()] = row.values()
    df.loc[period] = pd.Series(row)
    
df['Payment Date'] = rng

In [ ]:
df

In [ ]:


# Calculate mortgage payments and mortgage costs


def calc_monthly_payment(P, nyear, rate_year_perc):
    """ Find the monthly payments for a loan.
    
    Parameters
    ----------
    P : float
      The loan amount (principle).
    nyear : float
      Length of the loan in years
    rate_year_perc : float
      The yearly interest rate as a percentage (e.g. 5 if 5%).
    
    """
    rate_month = rate_year_perc / 100. / 12.
    nmonth = nyear * 12.
    term = (1 + rate_month)**nmonth
    return P * rate_month * term / (term - 1)


def calc_amort(P, rate_year_perc, mp):
    """
    Calculate monthly amortizations, with and without an offset.
    
    Parameters
    ----------
    P : float
      Principle (the amount you borrow) in $
    rate_year_perc : float
      Yearly interest rate (percent)
    mp : float
      Monthly payment in $
    
    Returns
    -------
    M : dict
    """
    M = pd.Series()
    M['rate_year_perc'] = rate_year_perc
    M['rate_year'] = rate_year_perc / 100.
    M['rate_month'] = M['rate_year'] / 12.
    M['P'] = [P]
    M['P_off'] = [P]
    
    n = 0
    r = M['rate_month']
    paid = 0
    paidoff = 0
    while M['P'][-1] > 0: 
        dP = mp - M['P'][-1] * r
        payment = (mp if M['P'][-1] - dP > 0 else M['P'][-1])
        paid += payment
        M['P'].append(M['P'][-1] - dP)
        if M['P_off'][-1] > 0:
            dPoff = mp - M['P_off'][-1] * r
            p = M['P_off'][-1] - dPoff
            if p < 0:
                p = 0
            M['P_off'].append(p)
            payment = (mp if M['P_off'][-1] - dPoff > 0 else M['P_off'][-1])
            paidoff += payment
            
        n += 1
        if n > 1e6:
            s = 'Too many iterations!'
            raise RuntimeError(s)

    M['nmonth'] = n
    M['nyear'] = n / 12.
    M['total_paid'] = paid
    M['monthly_payment'] = mp
    return M


import numpy as np

RED = '#fe8082'
BLUE = '#8083fc'

def doplot(M, ax, max_year=None, lw=0):
    years = np.arange(len(M['P'])) / 12.
    years_off = np.arange(len(M['P_off'])) / 12.
    ax.fill_between(years, np.array(M['P'])*1e-3, color=RED, label='no offset', 
                 lw=lw)
    if M['offset'] > 0:
        ax.fill_between(years_off, np.array(M['P_off'])*1e-3, color=BLUE, 
                     lw=lw, label='${:,g} offset'.format(M['offset']))
    ax.set_ylim(0, M['P'][0]*1e-3)
    ax.set_xlim(0, years[-1])
    ax.set_xlabel('Time (years)')
    ax.set_ylabel('Outstanding amount ($K)')
    ax.legend(frameon=0, loc=0)
    
    # the return on investing at an interest rate of 'rate_year', to compare how much 
    # you would gain putting the amount in a normal bank account instead of the 
    # offset.
    returns = M['offset'] * (1 + M['rate_year'])**M['nyear_offset'] - M['offset']
    
    s = "Rate {:.3g}% ${:,.4g} per month".format(M['rate_year_perc'], M['monthly_payment'])
    puttext(0.03, 0.3, s, ax, ha='left', fontsize=8,fontweight='bold')
    s = "Total paid ${:,g}".format(M['total_paid'])
    puttext(0.03, 0.25, s, ax, ha='left', fontsize=8)
    if M['offset'] > 0:
        s = """Total paid (offset) ${:,g}
difference at {:.3g} yr is ${:,g}
${:,g} from instead investing offset amount 
  at {:g}% after {:.3g} yr
""".format(M['total_paid_offset'], M['nyear_offset'],
           M['total_paid'] - M['total_paid_offset'],
           returns, M['rate_year_perc'], M['nyear_offset']
           )
        puttext(0.03, 0.02, s, ax, ha='left', fontsize=8)
    
    ax.minorticks_on()

In [ ]:
# length of the loan in years
Nyear = 14

# the yearly percentage interest rate 
rate = 6 

# the amount borrowed
principle = 300000.0

# The amount in a 100% offset account
offset = 50000.0

# Start end years that the offset amount is present in the account
start, end = 0, None

In [ ]:
help(calc_amort)

In [ ]:
calc_amort(470000, 8, 4500)

In [ ]:
# plot two examples, one making the mininum repayment, another making a 
# constant repayment of $2000 per month.

fig = plt.figure(figsize=(8,8))
fig.clf()
ax = fig.add_subplot(221)
monthly_payment = calc_monthly_payment(principle, Nyear, rate)
M = calc_amort(principle, rate, monthly_payment, offset, 
               offset_start_year=start, offset_end_year=end)
doplot(M, ax)
ax = fig.add_subplot(222)
ax.set_xlim(0, Nyear)
monthly_payment = max(monthly_payment, 2000)
M = calc_amort(principle, rate, monthly_payment, offset, 
                   offset_start_year=start, offset_end_year=end)
doplot(M, ax)
ax.set_xlim(0, Nyear)

plt.show()

In [ ]:
fig.savefig('mortgage_plots.pdf', bbox_inches='tight')

In [ ]:
12 * 30